# Week 1 - Retreiving and Preparing Text for Machines

This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze. This notebook outlines scraping text from the
web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Finally, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis.

For this notebook we will be using the following packages:

We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. 

In [1]:
!pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package

  Cloning git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /private/var/folders/4b/br29cjkn1s3_7qxk9__kv8xm0000gn/T/pip-req-build-ze5ycfl8
  Running command git clone -q git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git /private/var/folders/4b/br29cjkn1s3_7qxk9__kv8xm0000gn/T/pip-req-build-ze5ycfl8


  Created wheel for lucem-illud: filename=lucem_illud-8.0.1-py3-none-any.whl size=34913 sha256=34bde6acef2d16e2029eb91b7e2741437cc610301afd34b01121f8533ee4fc0b
  Stored in directory: /private/var/folders/4b/br29cjkn1s3_7qxk9__kv8xm0000gn/T/pip-ephem-wheel-cache-fdlk6qwx/wheels/a5/6a/0f/ddf40c68215bc4b072c455caa6a08e0031c92187c93a1b5a81
Successfully built lucem-illud


If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells. 

In [1]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [2]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [3]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [4]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"X-spLgpAICEAAKaLicQAAAAF","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":991020438,"wgRevisionId":991020438,"wgArticleId":473317,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles needing expert attention with no reason or talk parameter","Articles needing expert attention from April 2008","All articles needing expert attention","Sociology ar

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [5]:
 dir(wikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains. 



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [6]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dm


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [7]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag. 

In [8]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[1:10]:
    print(pTag.text)

Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of texts or artifacts which are assigned labels (sometimes called codes) to indicate the presence of interesting, meaningful pieces of content.[2][3] By systematically labeling the content of a set of texts, researchers can analyse patterns of content quantitatively using statistical methods, or use qualitative methods to analyse meanings of content within texts.

Computers are increasingly used in content analysis to automate the labeling (or coding) of documents. Simple computational techniques can provide descriptive data such as word frequencies and document lengths. Machine learning classifiers can greatly increase the number of texts that can be labeled, but the scientific utility of doing so is a matter of debate. Further, numerous computer-aided text analysis (CATA) computer programs are available that analyze text for pre-determined linguistic, s

Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [9]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

print(contentParagraphsDF[1:4])

                                       paragraph-text
0   Content analysis is the study of documents and...
1   Practices and philosophies of content analysis...
2   Computers are increasingly used in content ana...
3   Content analysis is best understood as a broad...
4   The simplest and most objective form of conten...
5   A further step in analysis is the distinction ...
6   Quantitative content analysis highlights frequ...
7   Siegfried Kracauer provides a critique of quan...
8   More generally, content analysis is research u...
9   By having contents of communication available ...
10  Computer-assisted analysis can help with large...
11  Robert Weber notes: "To make valid inferences ...
12  There are five types of texts in content analy...
13  Over the years, content analysis has been appl...
14  In recent times, particularly with the advent ...
15  Quantitative content analysis has enjoyed a re...
16  Content analysis can also be described as stud...
17  Manifest content is read

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators. 

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [10]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [11]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [12]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <span style="color:red">Section 1</span>
<span style="color:red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</span>

I will be scraping data from a thread on a fan forum for the TV show *Doctor Who*.

In [13]:
dw_forum = 'https://www.thedoctorwhoforum.com/forums/topic/revolution-of-the-daleks/'
requests.get(dw_forum)

<Response [200]>

In [14]:
dw_contentrequest = requests.get(dw_forum)
dwsoup = bs4.BeautifulSoup(dw_contentrequest.text, 'html.parser')
print(dwsoup.text[0:500])









Revolution of the Daleks | The Doctor Who Forum














			window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/2.3\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/2.3\/svg\/","svgExt":".svg","source":{"concatemoji":"https:\/\/www.thedoctorwhoforum.com\/wp-includes\/js\/wp-emoji-release.min.js?ver=4.8.15"}};
			!function(a,b,c){function d(a){var b,c,d,e,f=String.fromCharCode;if(!k||!k.fillText)return!1;switch(k.clearRect(0,0,j.width


In [15]:
dwsave = 'dw_forum.html'
with open(dwsave, mode='w', encoding='utf-8') as f:
    f.write(dw_contentrequest.text)

I searched for `<div>` tags rather than `<p>` tags in order to include usernames and time stamps in addition to the forum posts themselves.

In [16]:
#dwPTags = dwsoup.body.findAll('p')
#for pTag in dwPTags[1:15]:
#    print(pTag.text)

dwDivTags = dwsoup.body.findAll('div')
for divTag in dwDivTags[1:10]:
    print(divTag.text)








Search Forums:











Theories even more insane than what's actually happening







Home
Forums
Blogs
Activity
Etiquette
Members
Join
Help











Search Forums:











Theories even more insane than what's actually happening






Search Forums:
















Home
Forums
Blogs
Activity
Etiquette
Members
Join
Help









Home
Forums
Blogs
Activity
Etiquette
Members
Join
Help



Home
Forums
Blogs
Activity
Etiquette
Members
Join
Help






Revolution of the Daleks

Home › Forums › Episodes › The Thirteenth Doctor › Revolution of the Daleks
This topic contains 38 replies, has 17 voices, and was last updated by  Devilishrobby 1 day, 3 hours ago.



		Viewing 39 posts - 1 through 39 (of 39 total)
	





Author


			
				Posts
				
				
			
		




30 December 2020 at 18:47
#71265





Craig
@craig
Emperor



Happy New Year and welcome back to Doctor Who. It’s kinda been a while. Thank you to those who have been using the site, and my apologies for being a bit absent f

Using regular expressions, I'll now extract data from this page: each post's ID number, username, handle, date, and time. Then, I compile those into a dataframe.

In [17]:
findPostNumbers = '#(\d{5})'
dwPostNumbers = re.findall(findPostNumbers, dwsoup.body.text)

findName = '([A-Za-z0-9 ]+)\n@[A-Za-z0-9]+'
dwNames = re.findall(findName, dwsoup.body.text)

findHandle = '[A-Za-z0-9 ]+\n@([A-Za-z0-9]+)'
dwHandles = re.findall(findHandle, dwsoup.body.text)

findDates = '(\d+ \D+ \d+) at \d+:\d+'
dwDates = re.findall(findDates, dwsoup.body.text)

findTimes = '\d+ \D+ \d+ at (\d+:\d+)'
dwTimes = re.findall(findTimes, dwsoup.body.text)

dwPostData = pandas.DataFrame({'Number': dwPostNumbers, 'Name': dwNames, 'Handle': dwHandles, 'Date' : dwDates, 'Time' : dwTimes})
print(dwPostData)

   Number                Name              Handle              Date   Time
0   71265               Craig               craig  30 December 2020  18:47
1   71280       Devilishrobby       devilishrobby    1 January 2021  20:15
2   71281             Jamjarr             jamjarr    1 January 2021  20:21
3   71282   TranslatorCircuit   translatorcircuit    1 January 2021  23:41
4   71284      Dentarthurdent      dentarthurdent    2 January 2021  00:14
5   71286   TranslatorCircuit   translatorcircuit    2 January 2021  03:00
6   71288            MissRori            missrori    2 January 2021  03:39
7   71295          GalaxyMage          galaxymage    2 January 2021  04:47
8   71296              Davros              davros    2 January 2021  06:15
9   71297          Miapatrick          miapatrick    2 January 2021  06:50
10  71298          Miapatrick          miapatrick    2 January 2021  06:51
11  71299        BadWolfAlice        badwolfalice    2 January 2021  12:56
12  71300       Bluesquea

Finally, I extract the text of the posts themselves, using the fact that they are sandwiched between a handle and the date of the next post, and add this to the dataframe.

In [18]:
findPosts = '[A-Za-z0-9 ]+\n@[A-Za-z0-9]+\n?.+?\n?\n\n\n([\s\S]*?)(?:\d+ \D+ \d+ at)|(?:\s+Author\s+Posts\s+Viewing)'
dwPosts = re.findall(findPosts, dwsoup.body.text)
dwPostsDF = pandas.DataFrame({'Post' : dwPosts})
dwPostData['Text'] = dwPosts
dwPostData

,Number,Name,Handle,Date,Time,Text
0,71265,Craig,craig,30 December 2020,18:47,Happy New Year and welcome back to Doctor Who....
1,71280,Devilishrobby,devilishrobby,1 January 2021,20:15,"Well that was and wasn’t what I was expecting,..."
2,71281,Jamjarr,jamjarr,1 January 2021,20:21,Just seen it.\n \nTrying not to give spoilers....
3,71282,TranslatorCircuit,translatorcircuit,1 January 2021,23:41,This New Year’s special was yet another excell...
4,71284,Dentarthurdent,dentarthurdent,2 January 2021,00:14,@jamjarr Just to pick up on one point – ye...
5,71286,TranslatorCircuit,translatorcircuit,2 January 2021,03:00,"Craig, where is my long post which appeared he..."
6,71288,MissRori,missrori,2 January 2021,03:39,It didn’t need to be so long. Given how chunk...
7,71295,GalaxyMage,galaxymage,2 January 2021,04:47,I apologize in advance if this shows up multip...
8,71296,Davros,davros,2 January 2021,06:15,"For mine, this was a satisfying special. I man..."
9,71297,Miapatrick,miapatrick,2 January 2021,06:50,"Hi everyone!\nOK, so.\nYaz was great in this I..."



# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [19]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

[('https://en.wikipedia.org/wiki/Document', 0, 'documents'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Coding_(social_sciences)', 1, 'assigned labels (sometimes called codes)'), ('https://en.wikipedia.org/wiki/Semantics', 1, 'meaningful'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Quantitative_research', 1, 'quantitatively'), ('https://en.wikipedia.org/wiki/Statistics', 1, 'statistical methods'), ('https://en.wikipedia.org/wiki/Qualitative_research', 1, 'qualitative'), ('https://en.wikipedia.org/wiki/Text_(literary_theory)', 1, 'texts'), ('https://en.wikipedia.org/wiki/Machine_learning', 2, 'Machine learning')]


In [20]:
print(contentPTags)

[<p><b>Content analysis</b> is the study of <a href="/wiki/Document" title="Document">documents</a> and communication artifacts, which might be texts of various formats, pictures, audio or video. Social scientists use content analysis to examine patterns in communication in a replicable and systematic manner.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> One of the key advantages of using content analysis to analyse social phenomena is its non-invasive nature, in contrast to simulating social experiences or collecting survey answers.
</p>, <p>Practices and philosophies of content analysis vary between academic disciplines. They all involve systematic reading or observation of <a href="/wiki/Text_(literary_theory)" title="Text (literary theory)">texts</a> or artifacts which are <a href="/wiki/Coding_(social_sciences)" title="Coding (social sciences)">assigned labels (sometimes called codes)</a> to indicate the presence of interesting, <a href="/wiki/Semantic

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph. 

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [21]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

,paragraph-text,source,paragraph-number
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5
6,Quantitative content analysis highlights frequ...,https://en.wikipedia.org/wiki/Content_analysis,6
7,Siegfried Kracauer provides a critique of quan...,https://en.wikipedia.org/wiki/Content_analysis,7
8,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,8
9,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,9


Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [22]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [23]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF

,paragraph-text,source,paragraph-number,source-paragraph-number,source-paragraph-text
0,Content analysis is the study of documents and...,https://en.wikipedia.org/wiki/Content_analysis,0,None,None
1,Practices and philosophies of content analysis...,https://en.wikipedia.org/wiki/Content_analysis,1,None,None
2,Computers are increasingly used in content ana...,https://en.wikipedia.org/wiki/Content_analysis,2,None,None
3,Content analysis is best understood as a broad...,https://en.wikipedia.org/wiki/Content_analysis,3,None,None
4,The simplest and most objective form of conten...,https://en.wikipedia.org/wiki/Content_analysis,4,None,None
5,A further step in analysis is the distinction ...,https://en.wikipedia.org/wiki/Content_analysis,5,None,None
6,Quantitative content analysis highlights frequ...,https://en.wikipedia.org/wiki/Content_analysis,6,None,None
7,Siegfried Kracauer provides a critique of quan...,https://en.wikipedia.org/wiki/Content_analysis,7,None,None
8,"More generally, content analysis is research u...",https://en.wikipedia.org/wiki/Content_analysis,8,None,None
9,By having contents of communication available ...,https://en.wikipedia.org/wiki/Content_analysis,9,None,None



# <span style="color:red">Section 2</span>
<span style="color:red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</span>



I'll be extracting content from Archive of Our Own, a fanfiction website. First, I'm finding the URLs of all of the stories from the first page of results in the *Doctor Who* section.

In [24]:
fanfic_base = 'https://archiveofourown.org/'
requests.get('https://archiveofourown.org/tags/Doctor%20Who/works')

<Response [200]>

In [25]:
dwfanfic_request = requests.get('https://archiveofourown.org/tags/Doctor%20Who/works')
dwfanfic_soup = bs4.BeautifulSoup(dwfanfic_request.text, 'html.parser')
dwfanfic_save = 'dwfanfic_save.html'
with open(dwfanfic_save, mode='w', encoding='utf-8') as f:
    f.write(dwfanfic_request.text)

#I'm finding <h4> header tags in order to select fanfiction story titles

dwfanficH4Tags = dwfanfic_soup.body.findAll('h4')
for H4Tag in dwfanficH4Tags[1:15]:
    print(H4Tag.text)


Five times the Master rescued the Doctor, and one time they didn't
      by
        
      
starknight


Coffee Dates and Interruptions
      by
        
      
ughdotcom


One Shots first My Bi Boys
      by
        
      
KURTHUMMELSPOLYEVAN


Bolo
      by
        
      
Fushigikage


PODFIC - Double Crossing (Written by JessaLRynn)
      by
        
      
Julibellule



        for JessaLRynn


A Bottle of Bubbly
      by
        
      
TenRoseForeverandever


Peace In Smytheley
      by
        
      
Jaspre_Rose


Multi-Fandom Oneshots/Drabbles
      by
        
      
Himik0yumen0


Hope's Private Secretary
      by
        
      
Erinaceus87


Through The Looking Glass
      by
        
      
ElsaIsThereLifeOnMars


Monster Next Door
      by
        
      
ILikeToSneeze


Show Tunes
      by
        
      
FatalCookies


Barulho
      by
        
      
Fushigikage


Bad Boys: Rockin' All Over the World
      by
        
      
Sylvester7



In [26]:
dwfanficURLs = []
for linkNum, H4Tag in enumerate(dwfanficH4Tags):
    
    #I want hrefs that link to stories, which use relative links beginning with /works/
    
    atagLinks = H4Tag.findAll('a', href=re.compile('/works/'), class_=False)
    for aTag in atagLinks:
        relurl = aTag.get('href')
        linkText = aTag.text
        dwfanficURLs.append((
            urllib.parse.urljoin(fanfic_base, relurl),
            linkNum,
            linkText,
        ))
print(dwfanficURLs[:10])

[('https://archiveofourown.org/works/28881933', 1, "Five times the Master rescued the Doctor, and one time they didn't"), ('https://archiveofourown.org/works/28881777', 2, 'Coffee Dates and Interruptions'), ('https://archiveofourown.org/works/28881591', 3, 'One Shots first My Bi Boys'), ('https://archiveofourown.org/works/28880919', 4, 'Bolo'), ('https://archiveofourown.org/works/28842549', 5, 'PODFIC - Double Crossing (Written by JessaLRynn)'), ('https://archiveofourown.org/works/28880487', 6, 'A Bottle of Bubbly'), ('https://archiveofourown.org/works/28853904', 7, 'Peace In Smytheley'), ('https://archiveofourown.org/works/28878054', 8, 'Multi-Fandom Oneshots/Drabbles'), ('https://archiveofourown.org/works/28616826', 9, "Hope's Private Secretary"), ('https://archiveofourown.org/works/28340223', 10, 'Through The Looking Glass')]


Now I create an empty dataframe in order to organize text from each of these stories. 

In [27]:
storiesDF = pandas.DataFrame()
storiesDF['text'] = ''
storiesDF['source'] = ''
storiesDF['paragraph-number'] = ''
storiesDF['source-story-number'] = ''
storiesDF['source-link-text'] = ''
print(storiesDF)

Empty DataFrame
Columns: [text, source, paragraph-number, source-story-number, source-link-text]
Index: []


Here, I look at one story to help determine what content should be extracted in the dataframe--the first 6 and last 6 `<p>` tags of the story do not contain story content, so they will be excluded in the following function.

In [28]:
r = requests.get('https://archiveofourown.org/works/28616826/chapters/70138797')
hopesoup = bs4.BeautifulSoup(r.text, 'html.parser')

hopesoupPTags1 = hopesoup.body.findAll('p')[:10]
Test1 = pandas.DataFrame({'First 10 Paragraphs': hopesoupPTags1})
print(Test1)

hopesoupPTags2 = hopesoup.body.findAll('p')[-10:]
Test2 = pandas.DataFrame({'Last 10 Paragraphs': hopesoupPTags2})
print(Test2)

                                 First 10 Paragraphs
0  [\n    While we've done our best to make the c...
1                                 [\n, [Log In], \n]
2                [\n, [[], Remember Me], \n, [], \n]
3  [\n, [Work Search:], \n, [], \n, [tip: katekyo...
4                                              [[ ]]
5  [\n, [[1. Heaven Sent], \n, [2. Déjà Vu], \n, ...
6  [Depressed, distraught, and dying, magical gir...
7  [Now given a second chance to help her friends...
8    [“Five girls. A Shared Destiny. Do your best.”]
9  [Those eight words echoed through her mind as ...
                                  Last 10 Paragraphs
0  [The onslaught of questions swimming through S...
1  [But as Sayaka made it outside, she discovered...
2  [The Mysterious Woman casually walked out of t...
3  [“Ohhhhhhhhhhhhhhhhhhhh, wonderful! How ‘bout ...
4  [\n, [Abyranss], , , [noa_artist], , , [mandy1...
5  [\n          This work's creator has chosen to...
6  [(Plain text with limited HTML , [[<span>?<

Now I use the provided function in order to add each story paragraph from the linked pages into the dataframe.

In [29]:
def getTextFromDWFic(targetURL, sourceParNum, sourceText):
    parsDict = {'source' : [], 'paragraph-number' : [], 'text' : [], 'source-story-number' : [],  'source-link-text' : []}    
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    
    #I exclude the first 6 and last 6 paragraph tags in order to include only actual story content
    
    paragraphssoup = soup.body.findAll('p')[6:-6]
    for parNum, pTag in enumerate(paragraphssoup):
        parsDict['text'].append(pTag.text)
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-story-number'].append(sourceParNum)
        parsDict['source-link-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

In [33]:
for urlTuple in dwfanficURLs[:3]:
    #ignore_index means the indices will not be reset after each append
    storiesDF = storiesDF.append(getTextFromDWFic(*urlTuple),ignore_index=True)
storiesDF[0:20]

,text,source,paragraph-number,source-story-number,source-link-text
0,In which the Master pretends they could ever c...,https://archiveofourown.org/works/28881933,0,1,"Five times the Master rescued the Doctor, and ..."
1,I've had this in the works for a while (read: ...,https://archiveofourown.org/works/28881933,1,1,"Five times the Master rescued the Doctor, and ..."
2,(See the end of the work for more notes.),https://archiveofourown.org/works/28881933,2,1,"Five times the Master rescued the Doctor, and ..."
3,"\nThe Master was not, as a rule, fond of Earth...",https://archiveofourown.org/works/28881933,3,1,"Five times the Master rescued the Doctor, and ..."
4,\nSo it was with a pinched nose and an apologe...,https://archiveofourown.org/works/28881933,4,1,"Five times the Master rescued the Doctor, and ..."
5,"\nHe poked his head out of his TARDIS, which h...",https://archiveofourown.org/works/28881933,5,1,"Five times the Master rescued the Doctor, and ..."
6,"\n“No, Miss Grant, I don’t have any idea eithe...",https://archiveofourown.org/works/28881933,6,1,"Five times the Master rescued the Doctor, and ..."
7,"\n“Well, you shouldn’t have sent him off on hi...",https://archiveofourown.org/works/28881933,7,1,"Five times the Master rescued the Doctor, and ..."
8,\n“It was the Doctor who didn’t want you going...,https://archiveofourown.org/works/28881933,8,1,"Five times the Master rescued the Doctor, and ..."
9,"\n“And it was you who listened to him. Oh, Bri...",https://archiveofourown.org/works/28881933,9,1,"Five times the Master rescued the Doctor, and ..."


Because Archive of Your Own story pages don't contain additional relevant links, I decided to use a different site for the recursive spider. I'll extract data from the *Doctor Who* fandom wiki.

In [44]:
#First I define the recursive spider function

def spider(url, link_set, depth):
    if depth == 0:
        return {url}
    
    else:
        soup = bs4.BeautifulSoup(requests.get(url).text, 'html.parser')
        
        #I limited the number of links found on a page to 20, and started 20 links in to avoid menu/navigation items
        #That can be easily adjusted
        
        aTags = soup.body.findAll('a', href=re.compile('/wiki/'), class_=False)[20:40]
        for a in aTags:
            link = urllib.parse.urljoin(root_url, a.get('href'))
            link_set.add(link)
            link_set = link_set | spider(link, link_set, depth-1)
    
    return link_set

In [45]:
root_url = 'https://tardis.fandom.com/wiki/Doctor_Who'
link_set = set()

#Here I am running the function with a depth of 2--also adjustable

link_set = spider(root_url, link_set, 2)
dwwikiURLs = list(link_set)
dwwikiURLs[:20]

['https://tardis.fandom.com/wiki/Torchwood_comic_stories',
 'https://tardis.fandom.com/wiki/K9_and_Company',
 'https://tardis.fandom.com/wiki/Gallifrey_(audio_series)',
 'https://tardis.fandom.com/wiki/The_Daleks_(comic_series)',
 'https://tardis.fandom.com/wiki/Category:Stories',
 'https://tardis.fandom.com/wiki/Virgin_Missing_Adventures',
 'https://tardis.fandom.com/wiki/Obverse_Books',
 'https://tardis.fandom.com/wiki/The_Companion_Chronicles',
 'https://tardis.fandom.com/wiki/BBC_Eighth_Doctor_Adventures',
 'https://tardis.fandom.com/wiki/BBC_New_Series_Adventures',
 'https://tardis.fandom.com/wiki/The_Lost_Stories',
 'https://tardis.fandom.com/wiki/Faction_Paradox_(audio_series)',
 'https://tardis.fandom.com/wiki/Doctor_Who%3F_(comic_series)',
 'https://tardis.fandom.com/wiki/Big_Finish_Productions',
 'https://tardis.fandom.com/wiki/Jago_and_Litefoot',
 'https://tardis.fandom.com/wiki/Bernice_Summerfield_(series)',
 'https://tardis.fandom.com/wiki/Virgin_New_Adventures',
 'https:/

Now I extract text from these URLs.

In [47]:
dwwikiDF = pandas.DataFrame()
dwwikiDF['text'] = ''
dwwikiDF['source'] = ''
dwwikiDF['paragraph-number'] = ''

def getTextFromdwwiki(targetURL):
    parsDict = {'source' : [], 'paragraph-number' : [], 'text' : []}    
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    
    paragraphssoup = soup.body.findAll('p')
    for parNum, pTag in enumerate(paragraphssoup):
        parsDict['text'].append(pTag.text)
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
    return pandas.DataFrame(parsDict)

#I'm scraping the first 6 URLs here

for urlTuple in dwwikiURLs[:6]:
    dwwikiDF = dwwikiDF.append(getTextFromdwwiki(urlTuple),ignore_index=True)
dwwikiDF[0:20]

,text,source,paragraph-number
0,Torchwood comic stories appeared in Torchwood ...,https://tardis.fandom.com/wiki/Torchwood_comic...,0
1,"K9 and Company (TV series), as this page needs...",https://tardis.fandom.com/wiki/K9_and_Company,0
2,Talk about it here.,https://tardis.fandom.com/wiki/K9_and_Company,1
3,\n\nK9 and Company\n\n\n\n\n\n\nPublisher:\nBB...,https://tardis.fandom.com/wiki/K9_and_Company,2
4,K9 and Company was a proposed television serie...,https://tardis.fandom.com/wiki/K9_and_Company,3
5,The series was to have starred Elisabeth Slade...,https://tardis.fandom.com/wiki/K9_and_Company,4
6,The pilot episode was produced by then-Doctor ...,https://tardis.fandom.com/wiki/K9_and_Company,5
7,"Although the pilot did not result in a series,...",https://tardis.fandom.com/wiki/K9_and_Company,6
8,The pilot episode was titled A Girl's Best Fri...,https://tardis.fandom.com/wiki/K9_and_Company,7
9,The pilot episode saw Sarah discovering a boxe...,https://tardis.fandom.com/wiki/K9_and_Company,8


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [48]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3707,"posts-type":false,"posts":[{"id":"639732477227892736","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/639732477227892736","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/639732477227892736\/cat-demands-hairspray-sniffs","type":"photo","date-gmt":"2021-01-08 09:00:32 GMT","date":"Fri, 08 Jan 2021 10:00:32","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1610096432,"format":"html","reblog-key":"ZbrAwRC4","slug":"cat-demands-hairspray-sniffs","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"639732477227892736\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_639732477227892736\"><iframe id=\"like_iframe_639732477227892736\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [49]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [50]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [51]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,639732477227892736,https://64.media.tumblr.com/31e8882b2fd934f7b9...,"Fri, 08 Jan 2021 10:00:32","[cat, cats, lol, lolcat, lolcats]",jpg
1,639724919901585408,https://64.media.tumblr.com/fb868dea41853ad734...,"Fri, 08 Jan 2021 08:00:25","[cat, cats, lol, lolcat, lolcats]",jpg
2,639717378220015616,https://64.media.tumblr.com/61e3d59b36423880a9...,"Fri, 08 Jan 2021 06:00:33","[cat, cats, lol, lolcat, lolcats]",jpg
3,639709877361688576,https://64.media.tumblr.com/12f20ceaaa85f5a91e...,"Fri, 08 Jan 2021 04:01:19","[cat, cats, lol, lolcat, lolcats]",jpg
4,639702313021440000,https://64.media.tumblr.com/2386aff8793f75dcd7...,"Fri, 08 Jan 2021 02:01:06","[cat, cats, lol, lolcat, lolcats]",jpg
5,639687179569889280,https://64.media.tumblr.com/61e3d59b36423880a9...,"Thu, 07 Jan 2021 22:00:33","[cat, cats, lol, lolcat, lolcats]",jpg
6,639679639402479617,https://64.media.tumblr.com/61e3d59b36423880a9...,"Thu, 07 Jan 2021 20:00:42","[cat, cats, lol, lolcat, lolcats]",jpg
7,638101724326936576,https://64.media.tumblr.com/653968fb91a4356937...,"Mon, 21 Dec 2020 10:00:25","[cat, cats, lol, lolcat, lolcats]",png
8,637014579763871744,https://64.media.tumblr.com/e649f37db75d7901b9...,"Wed, 09 Dec 2020 10:00:43","[cat, cats, lol, lolcat, lolcats]",png
9,637007022068498432,https://64.media.tumblr.com/9e5288c65670863e89...,"Wed, 09 Dec 2020 08:00:36","[cat, cats, lol, lolcat, lolcats]",jpg


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [52]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [53]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory 

In [54]:
with open('./data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

, and Train.]

PUCK
  If we shadows have offended,
  Think but this,--and all is mended,--
  That you have but slumber'd here
  While these visions did appear.
  And this weak and idle theme,
  No more yielding but a dream,
  Gentles, do not reprehend;
  If you pardon, we will mend.
  And, as I am an honest Puck,
  If we have unearned luck
  Now to 'scape the serpent's tongue,
  We will make amends ere long;
  Else the Puck a liar call:
  So, good night unto you all.
  Give me your hands, if we be friends,
  And Robin shall restore amends.

[Exit.]





End of Project Gutenberg Etext of A Midsummer Night's Dream by Shakespeare
PG has multiple editions of William Shakespeare's Complete Works



By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory. 

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [55]:
targetDir = './data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [56]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

,text
julius_caesar.txt,"Dramatis Personae\n\n JULIUS CAESAR, Roman st..."
as_you_like_it.txt,AS YOU LIKE IT\n\nby William Shakespeare\n\n\n...
tempest.txt,"The Tempest\n\nActus primus, Scena prima.\n\nA..."
phoenix_and_the_turtle.txt,THE PHOENIX AND THE TURTLE\n\nby William Shake...
king_lear.txt,The Tragedie of King Lear\n\n\nActus Primus. S...
passionate_pilgrim.txt,THE PASSIONATE PILGRIM\n\nby William Shakespea...
cymbeline.txt,The Tragedie of Cymbeline\n\nActus Primus. Sco...
coriolanus.txt,THE TRAGEDY OF CORIOLANUS\n\nby William Shakes...
two_gentlemen_of_verona.txt,THE TWO GENTLEMEN OF VERONA\n\nby William Shak...
rape_of_lucrece.txt,THE RAPE OF LUCRECE\n\nby William Shakespeare\...


Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [57]:
#information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.3
%���������
4 0 obj
<< /Length 5 0 R /Filter /FlateDecode >>
stream
x�]۶�F�}�W�c����T���C�i�<t�bM�f
Tn�<3_��CDf��J�N�i��#�%.;.���	?߄��7�]8������ux��}޾m����y��bǾ����!���$�Ǯ���C��F�����p���5��1��1�P<�{�$��/$�P�s�v��PgH?�����Q�~�*�:l��ˇ�m�ǰ��C�l����܊��E�����!�^�y��m�$�Ý���wۡل׼�6w���ī�K�~؞���r��~?�ˡkO�;6IH�9{ԡ��� ]?�E�E��~���.l������+��W�_�����C��S�|�~C��N�3ӛB`8�ޚj9���AZ��0�d�l^������SY	�Ƨ��>q�ۇ&
����.�����0����; ��>a8�$w�p��p����ST���. �7��@����)��&1�|���WՃ jOv�G2b�L8I��N�@gǍ������O�C��������IN@@�����}�8��+L����a�&ү�o�V(���0���+5�
SfS&��<�2�����>l�V��&��=4⇤=�W��<�JMo����"����d�C����[vY�|K{_ܔ\����%H�/@'�QA�+D�l��c��L�G�.��	�̎�V�:f>���AwK���o$`D��bE45�0�%th6h�����>*�2vQd�+M��Y}�Q���u�[���N�o'b��/u�.r'Z���J�e8�v��;��{T�	�����^8�  l<�E�<���b�����C8j��f��xB>K�����|w��f�|?�s̭��Y�'�Ip&�"�A���f�?�!IYi���U�"��y;���#��e3)�+B�&����<E9I�g�/]"D��yfC;e����Y^�z ��s'�)/�X�-HY��<ˬ�ݰ

It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [58]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [59]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [60]:
print(readPDF(infoExtractionBytes)[:550])

Department of  Sociology 

THE UNIVERSITY OF CHICAGO 

SOCIOLOGY 40133 

Computational Content Analysis 

Friday 1:00 – 3:50pm 
Winter 2017-2018 
Classroom: Harper Memorial 130       
http://chalk.uchicago.edu/ 

 

                                                                                           

          Office: McGiffert 210 
                                                    Tel.: 834-3612; jevans@uchicago.edu 
                                  Office Hours: Thursday 12:30-2:30pm 

     

        James A. Evans            

    


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [61]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud_2020` so we can use it later without having to retype it.

In [62]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [63]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

File is not a zip file


We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [64]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

 
 

Accessing the Research Computing Center Resources

To connect to the midway compute cluster to access your home directory and the macs60000 storage space, and utilize the HPC resources, you will either use a terminal client (with or without X11 forwarding capabilities) or the Linux remote desktop server software client (Thinlinc) to connect to the midway cluster. To submit jobs, monitor jobs, browse directories or do other computing you will need to connect through either the terminal or remote desktop. Setup and utilization of these clients will be discussed below in the context of your local platform’s architecture.
SSH Client Setup & Remote Desktop Server


Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.


# <span style="color:red">Section 3</span>
<span style="color:red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</span>


I will extract the text of Professor Evans's CV.

In [65]:
CVpdf = 'https://sociology.uchicago.edu/sites/sociology.uchicago.edu/files/James%20Evans%20CV20.pdf'
CVRequest = requests.get(CVpdf, stream=True)

def readPDF(pdfFile):
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

CVExtractionBytes = io.BytesIO(CVRequest.content)
print(readPDF(CVExtractionBytes)[:1000])

J​AMES​ A​LLEN​ E​VANS 

 

Department of Sociology, University of Chicago, Chicago IL 60637 
Santa Fe Institute, Santa Fe, NM 87501 
Phone: 773.834-3612 (O); 773.324.1393 (H).  Fax: 773.702.4849.   
Email: ​jevans@uchicago.edu  
ORCiD ID: 0000-0001-9838-0707 

 

University of Chicago. 

 
P​ROFESSIONAL​ E​XPERIENCE
2015— Founding Faculty Director, Computational Social Science M.A. Program​, 
2013— Founding Director, Knowledge Lab​, University of Chicago. 
2016— Professor. Department of Sociology​, ​Committee on Conceptual and Historical 
Studies of Science, and the College​, University of Chicago. 
2012— Associate Professor (with tenure). Department of Sociology, Committee on 
Conceptual and Historical Studies of Science, and the College, University of 
Chicago. 

2004-12  Assistant Professor. Department of Sociology, Committee on Conceptual and 

Historical Studies of Science, and the College, University of Chicago.  

Instructor. Department of Sociology, Stanford University

2005-0

I'll now organize each publication from the CV into a dataframe using regular expressions.

In [67]:
CVread = readPDF(CVExtractionBytes)

findPublication = '\n\d\d?\. (.+)'
CVPublication = re.findall(findPublication, CVread)

CVPublications = pandas.DataFrame({'Publication': CVPublication})
print(CVPublications)

                                          Publication
0   Wang, Rose E., Sarah A. Wu, James Evans, Joshu...
1   Kang, Donghyun and James Evans. forthcoming. “...
2   Wu, Lingfei, Linzhuo Li, and James Evans. 2020...
3   Yin, Yian, Yang Wang, James Evans & Dashun Wan...
4   Evans, James and Jacob Foster. 2019. “Computat...
5   Kozlowski, Austin, Matt Taddy and James Evans....
6   Danchev, Valentin, Andrey Rzhetsky & James Eva...
7   Sengupta, Nandana, Nathan Srebro & James A. Ev...
8   Wu, Lingfei, Dashun Wang & James A. Evans. 201...
9   McMahan, Peter & James A. Evans. 2018. “Ambigu...
10  Shi, Feng, Misha Teplitskiy, Eamon Duede, Jame...
11  Asoudeh, Shahab, Tingran Gao & James Evans. 20...
12  Asoudeh, Shahab, Tingran Gao & James Evans. 20...
13  Börner, Katy, Olga Scrivner, Mike Gallant, Shu...
14  Azoulay, Pierre, Joshua Graff-Zivin, Brian Uzz...
15  Teplitskiy, Misha, Daniel Acuna, Aida Elamrani...
16  Gerow, Aaron, Yuening Hu, Jordan Boyd-Graber, ...
17  Katariya, Sumeet, Lalit 

Next, I'm organizing Professor Evans's work and educational experiences into a dataframe by searching for each date or range of dates at the start of a new line, followed by the text that comes after those dates.

In [68]:
findExperienceDate = '\n(\d{4}—?-?\d?\d?) +.+'
CVExperienceDates = re.findall(findExperienceDate, CVread)

findExperience = '\n\d{4}—?-?\d?\d? +(.+)'
CVExperience = re.findall(findExperience, CVread)

CVTimeline = pandas.DataFrame({'Date': CVExperienceDates, 'Experience': CVExperience})
print(CVTimeline[1:50])

       Date                                         Experience
1     2013—  Founding Director, Knowledge Lab​, University ...
2     2016—  Professor. Department of Sociology​, ​Committe...
3     2012—  Associate Professor (with tenure). Department ...
4   2004-12  Assistant Professor. Department of Sociology, ...
5   2005-08  Associate Editor, ​American Journal of Sociolo...
6   1995-97  Research Associate. Graduate School of Busines...
7   1992-93  Co-founder and Executive Director. Utah High S...
8     2018—       External Faculty Member, Santa Fe Institute 
9     2017—  Visiting Faculty, Computer Science, Tsinghua U...
10    2015—  External Faculty Member, NICO—Northwestern Ins...
11    2015—  Affiliate, METRICS—Meta-Research Innovation Ce...
12  1999-04  Ph.D. in Sociology. Stanford University. Commi...
13  1997-99          M.A. in Sociology.  Stanford University. 
14  2002-04  Social Science Research Council (SSRC) Fellow,...
15  2002-05  SSRC Fellow, Program on Philanthropy and t

Then I'll organize those into chronological order to create a timeline.

In [69]:
CVTimeline.sort_values(by=['Date'], inplace=True)
print(CVTimeline[1:50])

       Date                                         Experience
7   1992-93  Co-founder and Executive Director. Utah High S...
40  1993-94  Teaching Assistant to F. Kent Nielsen, “Histor...
18    1994—     Golden Key and Phi Kappa Phi Honor Societies. 
38  1995-96  Research Associate to V.G. Narayanan, Harvard ...
37  1995-97  Research Associate to George Wu, Harvard Busin...
36  1995-97  Research Associate to Kathleen McGinn, Harvard...
6   1995-97  Research Associate. Graduate School of Busines...
75  1996-04  Member, Academy of Management Association: Org...
39  1996-97  Teaching Assistant to Kathleen Valley, “Negoti...
17  1997-98  Stanford Graduate Fellow in the Department of ...
13  1997-99          M.A. in Sociology.  Stanford University. 
74    1997—  Member, American Sociological Association: Sci...
34  1998-01  Research Assistant to Stephen Barley, Stanford...
35  1998-99  Research Assistant to Mark Granovetter, Stanfo...
71  1999-00  Graduate Admissions Committee Member, Stan

### Other sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

